In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from vehicle import Vehicle, State, Control
from utils import circular_trajectory, stadium_trajectory, show_animation
from pid import PidController, PidPathFollower

import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from matplotlib.lines import Line2D
from IPython import display

from tqdm.notebook import tqdm

In [ ]:
vehicle = Vehicle()
states = [vehicle.state.copy()]

num_steps = 100    
for i in range(num_steps):
    states.append(vehicle.update(u={"steer": 0.6 * np.sin(i/3.0) + 0.2, "accel": 2.2 * np.sin(i/10.0)}))

df = pd.DataFrame(states)
plt.plot(df["x"], df["y"]);
plt.axis("equal");

# show_animation(vehicle, interval_ms=delta_t*1000)
# vehicle.save_animation("vehicle.mp4", interval=int(delta_t * 1000), movie_writer="ffmpeg")

In [ ]:
def simulate(init_state: State, control_func, duration: float, dt: float):
    vehicle = Vehicle(init_state=init_state, dt=dt)

    states = [vehicle.state]
    controls = [{"steer": 0.0, "accel": 0.0}]

    num_steps = int(duration / dt)
    for _ in tqdm(range(num_steps)):
        control: Control = control_func(vehicle.state)
        state = vehicle.update(control)
        
        states.append(state)
        controls.append(control)

    return pd.DataFrame(states), pd.DataFrame(controls)

In [ ]:
duration = 100
dt = 0.1

reference = circular_trajectory()
init_state = dict(reference.iloc[0])
init_state["v"] = 0.0

Kp_steer = 1.0
Ki_steer = 0.8
Kd_steer = 0.0

Kp_accel = 0.4
Ki_accel = 0.0
Kd_accel = 0.0

pid_controller = PidPathFollower(reference, dt, Kp_steer, Ki_steer, Kd_steer, Kp_accel, Ki_accel, Kd_accel)

states, controls = simulate(init_state, pid_controller, duration, dt)

In [ ]:
plt.plot(reference["x"], reference["y"])
plt.plot(states["x"], states["y"])
plt.axis("equal");

In [ ]:
show_animation(reference, states.iloc[::75], controls[::75], vehicle.max_steer_abs, vehicle.max_accel_abs, 100)

In [ ]:

    # # --- Plotting Results ---
    # plt.style.use('seaborn-v0_8-darkgrid')
    # # Make the trajectory plot larger to better see the arrows
    # fig, axs = plt.subplots(6, 1, figsize=(10, 10), gridspec_kw={'height_ratios': [3, 1, 1, 1, 1, 1]})

    # # Plot trajectory paths
    # axs[0].plot(ref_x, ref_y, 'r--', label="Reference Trajectory")
    # axs[0].plot(x_hist, y_hist, 'b-', label="Vehicle Path")
    
    # arrow_length = 2.0  # Visual length of the heading arrows
    
    # # Add arrows for reference heading
    # # We draw fewer arrows than data points to keep the plot clean
    # for i in range(0, len(ref_x), ARROW_SKIP):
    #     axs[0].arrow(ref_x[i], ref_y[i],
    #                  arrow_length * np.cos(ref_yaw[i]), arrow_length * np.sin(ref_yaw[i]),
    #                  head_width=0.8, head_length=1.0, fc='deeppink', ec='deeppink', alpha=0.7)

    # # Add arrows for vehicle heading
    # for i in range(0, len(x_hist), ARROW_SKIP):
    #     axs[0].arrow(x_hist[i], y_hist[i],
    #                  arrow_length * np.cos(yaw_hist[i]), arrow_length * np.sin(yaw_hist[i]),
    #                  head_width=0.8, head_length=1.0, fc='black', ec='black', alpha=0.9)


    # axs[0].set_title("Vehicle Trajectory Following with Heading Arrows")
    # axs[0].set_xlabel("X [m]")
    # axs[0].set_ylabel("Y [m]")
    
    # # Create a custom legend to represent paths and arrows
    # legend_elements = [Line2D([0], [0], color='r', linestyle='--', label='Reference Trajectory'),
    #                    Line2D([0], [0], marker='>', color='w', markerfacecolor='deeppink', markersize=10, label='Reference Heading'),
    #                    Line2D([0], [0], color='b', linestyle='-', label='Vehicle Path'),
    #                    Line2D([0], [0], marker='>', color='w', markerfacecolor='cyan', markersize=10, label='Vehicle Heading')]
    # axs[0].legend(handles=legend_elements)
    # axs[0].axis("equal")

    # # Plot velocity
    # axs[1].plot(time_hist, v_hist, label="Vehicle Speed")
    # axs[1].plot(time_hist, [ref_v[np.argmin(np.hypot(ref_x - x, ref_y - y))] for x, y in zip(x_hist, y_hist)], 'r--', label="Target Speed")
    # axs[1].set_title("Velocity Profile")
    # axs[1].set_xlabel("Time [s]")
    # axs[1].set_ylabel("Speed [m/s]")
    # axs[1].legend()

    # # Plot heading
    # axs[2].plot(time_hist, np.rad2deg(yaw_hist), label="Vehicle Heading")
    # axs[2].plot(time_hist, np.rad2deg([ref_yaw[np.argmin(np.hypot(ref_x - x, ref_y - y))] for x, y in zip(x_hist, y_hist)]), 'r--', label="Target Heading")
    # axs[2].set_title("Heading Profile")
    # axs[2].set_xlabel("Time [s]")
    # axs[2].set_ylabel("Heading [deg]")
    # axs[2].legend()

    # axs[3].plot(time_hist, steer_hist, label="Steering")
    # axs[3].set_title("Steering")
    # axs[3].set_xlabel("Time [s]")
    # axs[3].legend()    

    # axs[4].plot(time_hist, accel_hist, label="Acceleration")
    # axs[4].set_title("Acceleration")
    # axs[4].set_xlabel("Time [s]")
    # axs[4].legend()    

    # r, a, i, sin_val, hypot, dx, dy, cte, ang_err = zip(*debug_values)
    # max_val = 1000#100*np.pi
    # min_val = -max_val
    # axi = 5
    # # axs[axi].plot(time_hist[1:], np.clip(r, min_val, max_val), label='ang-ref')
    # # axs[axi].plot(time_hist[1:], np.clip(a, min_val, max_val), label='ang-vehicle')
    # # axs[axi].plot(time_hist[1:], np.clip(np.array(i)/len(ref_x), min_val, max_val), label='index')
    # # axs[axi].plot(time_hist[1:], np.clip(sin_val, min_val, max_val), label='sin')
    # # axs[axi].plot(time_hist[1:], np.clip(hypot, min_val, max_val), label='hypot')
    # # axs[axi].plot(time_hist[1:], np.clip(dx, min_val, max_val), label='dx')
    # # axs[axi].plot(time_hist[1:], np.clip(dy, min_val, max_val), label='dy')
    # axs[axi].plot(time_hist[1:], np.clip(cte, min_val, max_val), label='cte')
    # # axs[axi].plot(time_hist[1:], np.clip(ang_err, min_val, max_val), label='ang-err')
    # axs[axi].legend()

    # plt.tight_layout()
    # plt.show()
    # return vehicle